In [1]:
from scripts import support, insd, kuroda, nras, metrics, vectorization as v
import pandas as pd
import numpy as np

In [2]:
# Base matrix
df = pd.read_excel("./data/all2011.xlsx", 'М-имп',skiprows=1,index_col=2)
df = df.iloc[2:-2,2:-13]

In [3]:
# Predicted matrix
pred_df = pd.read_excel("./data/tri-2012.xlsx", 'М-имп',skiprows=1,index_col=2)
pred_df = pred_df.iloc[2:-2,2:-11]
shape = pred_df.shape

In [4]:
# Sums of rows\columns
sums_rows = pred_df.sum(axis=1).values.reshape(-1,1)
sums_cols = pred_df.sum(axis=0).values.reshape(-1,1)

In [5]:
# Vectorization of matrix
print(v.tovector(df.values))

[[33559]
 [12]
 [16]
 ...
 [0]
 [0]
 [0]]


In [6]:
# Alternative to vectorization.tovector - simplier for pandas dataframes
vect_a = df.unstack().values.reshape(-1,1)
v.tomatrix(vect_a, shape)

array([[33559, 301, 33, ..., 314, 1556, 0],
       [12, 33, 0, ..., 9, 18, 0],
       [16, 0, 593, ..., 18, 0, 0],
       ...,
       [6, 0, 4, ..., 6379, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [7]:
# Adding coefficients of elements in sums of rows
# matrix 3*3
G = np.empty((0,3*3),float)
v.addrows(G,(3,3))

array([[1., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 1.]])

In [8]:
# Add coefficients of constraints & merge column of constraints
c = np.append(sums_rows,sums_cols,axis=0)
G = np.empty((0,shape[0]*shape[1]),float)
G = v.addrows(G,shape)
G = v.addcolumns(G,shape)
G

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [9]:
# Run INSD
res_df = insd.insd(aa=vect_a,c=c, G=G)
pd.DataFrame(v.tomatrix(res_df)).round(3).head()

Academic license - for non-commercial use only
Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1


,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,35012.661,315.235,40.575,0.000,3.421,0.0,0.000,0.000,64548.756,33009.027,...,35.696,150.406,4773.123,1889.925,7337.560,69.136,285.435,347.134,1493.830,0.0
1,16.165,44.586,0.000,5.261,0.000,0.0,0.000,0.000,351.247,0.000,...,1.154,2.772,19.482,4.008,11.149,0.000,4.717,12.684,22.749,0.0
2,23.802,0.000,992.586,0.000,0.000,0.0,0.000,0.000,18647.784,0.000,...,18.119,3.053,15.156,2.953,85.909,0.000,6.852,27.897,0.000,0.0
3,84.373,0.559,4.448,6620.211,15.000,0.0,53.434,11.302,121.794,0.000,...,1.447,14.846,92.164,52.763,208.139,15.234,1.561,17.901,14.154,0.0
4,0.000,0.000,0.000,0.000,10156.644,0.0,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0


In [10]:
# Metrics
metrics.get_values(res_df,pred_df.values)

MAPE 3748.71
SWAD 0.347
WAPE 132.195


In [17]:
# NRAS
res_df = nras.nras(G=G,aa=vect_a,c=c,accuracy=0.001, limit =  100)
metrics.get_values(res_df,pred_df.values)

49
MAPE 3761.955
SWAD 0.347
WAPE 132.127


In [17]:
# Kuroda
res_df = kuroda.kuroda(G=G,aa=vect_a,c=c,mtype=3)
metrics.get_values(res_df,pred_df.values)

ERROR:root:Traceback (most recent call last):
  File "var.pxi", line 307, in gurobipy.Var.__truediv__
ZeroDivisionError: float division

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Aska/GitHub/Disaggregation/scripts/kuroda.py", line 58, in kuroda
    t.add(0.5 * (x[j] / c[i] - at[j] / czero[i]) * (x[j] / c[i] - at[j] / czero[i]) * w[i][j])
  File "var.pxi", line 309, in gurobipy.Var.__truediv__
gurobipy.GurobiError: Divisor must be a constant

ERROR:root:Traceback (most recent call last):
  File "/Users/Aska/GitHub/Disaggregation/scripts/metrics.py", line 55, in mape
    x = x.flatten()
AttributeError: 'int' object has no attribute 'flatten'

ERROR:root:Traceback (most recent call last):
  File "/Users/Aska/GitHub/Disaggregation/scripts/metrics.py", line 106, in swad
    x = x.flatten()
AttributeError: 'int' object has no attribute 'flatten'

ERROR:root:Traceback (most recent call last):
  File "/Users/Aska/GitH

MAPE -1
SWAD -1
WAPE -1


In [48]:
# Загружаем данные из одной несимметричной таблицы 
quadrant1, df, codes_industries, codes_products = support.load_Rosstat_non_sym("./data/tri-2012.xlsx", 'М-имп', quandrant2_columns = 11, quadrant3_rows = 2)

Обрабатываем данные из таблицы в формате Rosstat "ТАБЛИЦА ИСПОЛЬЗОВАНИЯ ИМПОРТНОЙ ПРОДУКЦИИ за 2012 год (млн. рублей)"


In [ ]:
# Сохраняем таблицы в файл постранично 
df_d[0].columns.name = "Отечественный выпуск0"
result_tables = {support.pack_name("Отечественный выпуск0",years,'2008'): df_d[0]}
support.save_to_excel("test_res.xlsx", years, codes, **result_tables)

In [44]:
support.pack_name("мяумя",'2020-21000','2019')

'мяумя за 2020-21000гг(2019)'